Given a sample

In [15]:
sample = {"context": "بوداپـِست پایتخت، بزرگترین و پرجمعیت‌ترین شهر کشور مجارستان و مرکز سیاسی، فرهنگی، تجاری، صنعتی و ترابری این کشور است. بوداپست از سه بخشِ اُبودا (بودای قدیم)، بودا (تپه ای و در کرانهٔ باختری دانوب)، و پست (خلاف بودا کاملاً تخت و سمت خاوری رود) تشکیل شده‌است. جمعیت بوداپست طبق آمار اول ژانویهٔ سال ۲۰۱۹ برابر با ۱٬۷۵۲٬۲۸۶ نفر بوده‌است. مساحت شهر حدود ۵۲۵ کیلومتر مربع و یک واحد مستقل در تقسیمات کشوری مجارستان است.", "question": "بوداپست در کدام کشور است؟", "target": ["مجارستان"]}

# prepare input prompt

In [16]:
from prompt import PERSIAN_ONE

In [17]:
sample_prompt = PERSIAN_ONE.format(context=sample['context'],question=sample['question'])

In [18]:
print(sample_prompt)


  در ادامه به شما یک متن فارسی و یک سوال نشان داده می شود. شما باید برای سوال یک پاسخ بنویسید. سعی کنید پاسخ های خود را تا حد ممکن کوتاه بدهید.

  نمونه:

  سوال:
  ویتامین ای را چه موقع استفاده کنیم؟

  جواب:
  برای رفع چین و چروک پیشانی و دیگر نشانه های پیری

  متن:
  '''بوداپـِست پایتخت، بزرگترین و پرجمعیت‌ترین شهر کشور مجارستان و مرکز سیاسی، فرهنگی، تجاری، صنعتی و ترابری این کشور است. بوداپست از سه بخشِ اُبودا (بودای قدیم)، بودا (تپه ای و در کرانهٔ باختری دانوب)، و پست (خلاف بودا کاملاً تخت و سمت خاوری رود) تشکیل شده‌است. جمعیت بوداپست طبق آمار اول ژانویهٔ سال ۲۰۱۹ برابر با ۱٬۷۵۲٬۲۸۶ نفر بوده‌است. مساحت شهر حدود ۵۲۵ کیلومتر مربع و یک واحد مستقل در تقسیمات کشوری مجارستان است.'''

  سوال:
  '''بوداپست در کدام کشور است؟'''

  جواب:
  


# Evaluate model's response

In [14]:
! pip install hazm

In [11]:
from hazm import Normalizer

In [8]:
import numpy as np

In [19]:
sample_GPT_response = """
مجارستان
"""

In [6]:
def preprocess_text(s,normalizer):

    def normalize(text):
        text.replace("پاسخ:","")
        return normalizer.normalize(text)

    def remove_punc_stopword(text):
        exclude = ["?",".","!","؟",":","،",")","(","..."]
        return "".join(ch for ch in text if ch not in exclude)

    return normalize(remove_punc_stopword(s))

def compute_exact_match(prediction, truth):
    return int(preprocess_text(prediction) == preprocess_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(i):
    """helper function that retrieves all possible true answers from a squad2.0 example"""

    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example -
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]

    return gold_answers

In [9]:
def calculate_f1(model_ans_list,correct_ans_list):
  normalizer = Normalizer()
  f1 = []
  for i in range(0,len(model_ans_list)):
    model_ans = preprocess_text(model_ans_list[i],normalizer)
    correct_ans =  [preprocess_text(text,normalizer) for text in correct_ans_list[i]]
    f1.append(max((compute_f1(model_ans, answer)) for answer in correct_ans))

  return np.array(f1).mean()

In [20]:
gpt_results = [sample_GPT_response]
true_ans = [sample['target']]
f1 = calculate_f1(gpt_results,true_ans)
print(f'average f1: {f1}')

average f1: 1.0
